In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import os
from random import choice
import shutil
import pathlib
from pathlib import Path
import itertools
import cv2
from PIL import Image

# Ordner für das Video anlegen
VideoPath ='/content/drive/MyDrive/Greifen/Test/video'
print(os.listdir(VideoPath))
for video in os.listdir(VideoPath):
      name, endung = video.split('.')
      if not os.path.isdir(filePath + name):
        os.makedirs("/content/drive/MyDrive/Greifen/Test/" + name ) 

        print('Ordner erstellt')
        print(name)
      else:
        print('Ordner besteht')


vidcap = cv2.VideoCapture('/content/drive/MyDrive/Greifen/Test/video/Greifen_2_130767.mp4')
success,image = vidcap.read()
count = 1
while success:    
      cv2.imwrite("/content/drive/MyDrive/Greifen/Test/video/Greifen_2_130767/Greifen_2_130767-%d.jpg" % count, image)
      success,image = vidcap.read()
      print('Read a new frame: ', success)
      count += 1


#AnzahlFrames = 0
#for path in pathlib.Path("/content/drive/MyDrive/MA_LL/Test_Video_Greifen_1_310426/Greifen_1_310426").iterdir():
#    if path.is_file():
#        AnzahlFrames += 1
#count = 1
#while count < (AnzahlFrames + 1):
#    image = Image.open('/content/drive/MyDrive/MA_LL/Test_Video_Greifen_1_310426/Greifen_1_310426/Greifen_1_310426-%d.jpg' % count)
#    new_image = image.resize((250, 250))
#    new_image.save('/content/drive/MyDrive/MA_LL/Test_Video_Greifen_1_310426/data_resized/Greifen_1_310426-%d.jpg' % count)
#    print(count)
#    print(new_image.size)
#    count += 1


['Greifen_2_130767.mp4']
Ordner erstellt
Greifen_2_130767
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a

In [28]:
# Erstelle Text Datei mit Vorhersage(Phase, None) pro Frame
# Erstelle Test Datei mit zeitlicher Prediction für Phase Greifen 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from PIL import Image, ImageFile
import os
import pathlib
from pathlib import Path
from torch.optim import lr_scheduler
import torch.nn.init as init
from torch.nn import DataParallel
import numpy as np
from __future__ import absolute_import, division, print_function

# Modell laden
class resnet_lstm(torch.nn.Module):
    def __init__(self):
        super(resnet_lstm, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.share = torch.nn.Sequential()
        self.share.add_module("conv1", resnet.conv1)
        self.share.add_module("bn1", resnet.bn1)
        self.share.add_module("relu", resnet.relu)
        self.share.add_module("maxpool", resnet.maxpool)
        self.share.add_module("layer1", resnet.layer1)
        self.share.add_module("layer2", resnet.layer2)
        self.share.add_module("layer3", resnet.layer3)
        self.share.add_module("layer4", resnet.layer4)
        self.share.add_module("avgpool", resnet.avgpool)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)
        self.lstm = nn.LSTM(2048, 512, batch_first=True)
        self.fcDropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, 7)

        init.xavier_normal_(self.lstm.all_weights[0][0])
        init.xavier_normal_(self.lstm.all_weights[0][1])
        init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = x.view(-1, 3, 224, 224)
        x = self.share.forward(x)
        x = x.view(-1, 2048)
       # x = x.view(-1, 10, 2048)
        self.lstm.flatten_parameters()
        y, _ = self.lstm(x)
        y = y.contiguous().view(-1, 512)
        # y = self.fcDropout(y)
        y = self.fc(y)
        return y

use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if use_gpu else "cpu")

model = resnet_lstm()
model.load_state_dict(torch.load('/content/drive/MyDrive/Greifen/Greifen2_run01/lstm_epoch_14_length_10_opt_1_mulopt_1_flip_1_crop_1_batch_50_train_9762_val_9468.pth'))
model = DataParallel(model)
model.to(device)

phase_key = ['None', 'Greifen']

# Input transformieren
img_transforms = transforms.Compose([
    transforms.Resize((250, 250)),                                  
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.3456, 0.2281, 0.2233], [0.2528, 0.2135, 0.2104]), 
    ]) 

# Prediction
count = 1
i = 1
k = 1
for element in os.listdir('/content/drive/MyDrive/Greifen/Test/Greifen_2_130767'):           
    img = Image.open("/content/drive/MyDrive/Greifen/Test/Greifen_2_130767/Greifen_2_130767-%d.jpg" %count)
    img = img_transforms(img).to(device)  
    img = torch.unsqueeze(img, 0)

    model.eval()
    prediction = F.softmax(model(img), dim=1)
    prediction = prediction.argmax()
    print('Frame: {:2d}'.format(count))
    print(phase_key[prediction])

    if phase_key[prediction] == 'Greifen':
        i += 1
    
    if i > 20 and phase_key[prediction] == 'Greifen':
        with open("/content/drive/MyDrive/Greifen/Test/Bericht_Greifen_2_130767.txt") as text_file:
            with open("/content/drive/MyDrive/Greifen/Test/Bericht_Greifen_2_130767.txt", "a") as text_file: #'a' Text wird bestehender Datei immer wieder angehängt 
                  #text = 'Frame ' + str(count) + ': ' + phase_key[prediction] + '\n'
                  #text = phase_key[prediction] + '\n'
                  text = 'Greifen\n'
                  text_file.write("%s" % text)
    else: 
        with open("/content/drive/MyDrive/Greifen/Test/Bericht_Greifen_2_130767.txt", "a") as text_file: #'a' Text wird bestehender Datei immer wieder angehängt 
            #text = 'Frame ' + str(count) + ': ' + 'None\n'
            text = 'None\n'
            text_file.write("%s" % text)
    count += 1  

# Text Datei für zeitliche Vorhersage 
total_lines = 0
greifen_count = 0
none_count = 0
with open('/content/drive/MyDrive/Greifen/Test/Bericht_Greifen_2_130767.txt', 'r') as file:
    begin = next((i for i, line in enumerate(file, 1) if line.rstrip() == 'Greifen'),0)
    for line in file: 
        if 'Greifen' in line:
            greifen_count += 1

with open('/content/drive/MyDrive/Greifen/Test/Bericht_Greifen_2_130767.txt') as file:
    total_lines = sum(1 for line in file)
#print(begin)
#print(greifen_count)
#print(total_lines)

start = round(begin/30,1)
ende = round((begin+greifen_count)/30,1)
länge = round((total_lines)/30,1)

print('Länge Video: {:.4f} sec'.format(länge))
print('Start Greifen: {:.4f} sec, Ende Greifen: {:.4f} sec'.format(start, ende))
       
with open('/content/drive/MyDrive/Greifen/Test/pred_time_Greifen_2_130767.txt', 'w') as text_file:
    text = 'Länge Video: ' + str(länge) + '\n' + 'Start Greifen: ' + str(start) + ' sec\n' + 'Ende Greifen: ' + str(ende) + ' sec'
    text_file.write("%s" % text)




/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Frame:  1
None
Frame:  2
Greifen
Frame:  3
Greifen
Frame:  4
Greifen
Frame:  5
Greifen
Frame:  6
None
Frame:  7
Greifen
Frame:  8
Greifen
Frame:  9
Greifen
Frame: 10
Greifen
Frame: 11
Greifen
Frame: 12
Greifen
Frame: 13
Greifen
Frame: 14
Greifen
Frame: 15
Greifen
Frame: 16
Greifen
Frame: 17
Greifen
Frame: 18
Greifen
Frame: 19
Greifen
Frame: 20
Greifen
Frame: 21
Greifen
Frame: 22
Greifen
Frame: 23
Greifen
Frame: 24
Greifen
Frame: 25
Greifen
Frame: 26
Greifen
Frame: 27
Greifen
Frame: 28
Greifen
Frame: 29
Greifen
Frame: 30
Greifen
Frame: 31
Greifen
Frame: 32
Greifen
Frame: 33
Greifen
Frame: 34
Greifen
Frame: 35
Greifen
Frame: 36
Greifen
Frame: 37
Greifen
Frame: 38
Greifen
Frame: 39
Greifen
Frame: 40
Greifen
Frame: 41
Greifen
Frame: 42
Greifen
Frame: 43
Greifen
Frame: 44
Greifen
Frame: 45
Greifen
Frame: 46
Greifen
Frame: 47
Greifen
Frame: 48
Greifen
Frame: 49
None
Frame: 50
None
Frame: 51
None
Frame: 52
Greifen
Frame: 53
None
Frame: 54
Greifen
Frame: 55
Greifen
Frame: 56
Greifen
Frame: 57
